## Applying Negative Binomial Regression

## Model Specification
- #### Dependent variable:
    - Y: The migration flow (between Region)
- ####  Independent variables:
    - ΔRisk (x₁): Difference in climate risk between the destination and the origin
    - ΔHousing (x₂): Difference in housing affordability between the destination and the origin
    - ΔUnemployment (x₃): Difference in unemployment rate between the destination and the origin

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# Disable scientific notation & add commas
pd.set_option('display.float_format', '{:,.2f}'.format)
# Show all columns without truncation
pd.set_option('display.max_columns', None)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
import pandas as pd
flow_data = pd.read_csv('../../data/final/flow_data_final.csv')

In [3]:
flow_data.shape

(88888, 410)

In [4]:
flow = flow_data['Destination_Code'].unique()
len(flow)

392

In [5]:
flow_data.head()

,Origin_Code,Destination_Code,Origin_Name,Destination_Name,Origin_State,Destination_State,Origin_Region,Destination_Region,Flow,Flow_MOE,Origin_Crosswalk2023_CBSA Code,Origin_Crosswalk2023_CBSA Title,Origin_Crosswalk2023_State Name,Origin_NRI_AVLN_EALPE,Origin_NRI_CFLD_EALPE,Origin_NRI_CWAV_EALPE,Origin_NRI_ERQK_EALPE,Origin_NRI_HAIL_EALPE,Origin_NRI_HWAV_EALPE,Origin_NRI_HRCN_EALPE,Origin_NRI_ISTM_EALPE,Origin_NRI_LNDS_EALPE,Origin_NRI_LTNG_EALPE,Origin_NRI_RFLD_EALPE,Origin_NRI_SWND_EALPE,Origin_NRI_TRND_EALPE,Origin_NRI_TSUN_EALPE,Origin_NRI_VLCN_EALPE,Origin_NRI_WFIR_EALPE,Origin_NRI_WNTW_EALPE,Origin_NRI_AVLN_EALB,Origin_NRI_CFLD_EALB,Origin_NRI_CWAV_EALB,Origin_NRI_ERQK_EALB,Origin_NRI_HAIL_EALB,Origin_NRI_HWAV_EALB,Origin_NRI_HRCN_EALB,Origin_NRI_ISTM_EALB,Origin_NRI_LNDS_EALB,Origin_NRI_LTNG_EALB,Origin_NRI_RFLD_EALB,Origin_NRI_SWND_EALB,Origin_NRI_TRND_EALB,Origin_NRI_TSUN_EALB,Origin_NRI_VLCN_EALB,Origin_NRI_WFIR_EALB,Origin_NRI_WNTW_EALB,Origin_NRI_AVLN_EXPPE,Origin_NRI_CFLD_EXPPE,Origin_NRI_CWAV_EXPPE,Origin_NRI_ERQK_EXPPE,Origin_NRI_HAIL_EXPPE,Origin_NRI_HWAV_EXPPE,Origin_NRI_HRCN_EXPPE,Origin_NRI_ISTM_EXPPE,Origin_NRI_LNDS_EXPPE,Origin_NRI_LTNG_EXPPE,Origin_NRI_RFLD_EXPPE,Origin_NRI_SWND_EXPPE,Origin_NRI_TRND_EXPPE,Origin_NRI_TSUN_EXPPE,Origin_NRI_VLCN_EXPPE,Origin_NRI_WFIR_EXPPE,Origin_NRI_WNTW_EXPPE,Origin_NRI_AVLN_EXPB,Origin_NRI_CFLD_EXPB,Origin_NRI_CWAV_EXPB,Origin_NRI_ERQK_EXPB,Origin_NRI_HAIL_EXPB,Origin_NRI_HWAV_EXPB,Origin_NRI_HRCN_EXPB,Origin_NRI_ISTM_EXPB,Origin_NRI_LNDS_EXPB,Origin_NRI_LTNG_EXPB,Origin_NRI_RFLD_EXPB,Origin_NRI_SWND_EXPB,Origin_NRI_TRND_EXPB,Origin_NRI_TSUN_EXPB,Origin_NRI_VLCN_EXPB,Origin_NRI_WFIR_EXPB,Origin_NRI_WNTW_EXPB,Origin_NRI_BUILDVALUE,Origin_NRI_POPULATION,Origin_NRI_AREA,Origin_Migration16to20_A_Inflow_Estimate,Origin_Migration16to20_A_Outflow_Estimate,Origin_Migration16to20_A_NetMigration_Estimate,Origin_Migration16to20_A_GrossMigration_Estimate,Origin_Migration16to20_A_Inflow_MOE,Origin_Migration16to20_A_Outflow_MOE,Origin_Migration16to20_A_NetMigration_MOE,Origin_Migration16to20_A_GrossMigration_MOE,Origin_HousingPI25_Metro Area,Origin_HousingPI25_GEOID.1,Origin_HousingPI25_1980,Origin_HousingPI25_1981,Origin_HousingPI25_1982,Origin_HousingPI25_1983,Origin_HousingPI25_1984,Origin_HousingPI25_1985,Origin_HousingPI25_1986,Origin_HousingPI25_1987,Origin_HousingPI25_1988,Origin_HousingPI25_1989,Origin_HousingPI25_1990,Origin_HousingPI25_1991,Origin_HousingPI25_1992,Origin_HousingPI25_1993,Origin_HousingPI25_1994,Origin_HousingPI25_1995,Origin_HousingPI25_1996,Origin_HousingPI25_1997,Origin_HousingPI25_1998,Origin_HousingPI25_1999,Origin_HousingPI25_2000,Origin_HousingPI25_2001,Origin_HousingPI25_2002,Origin_HousingPI25_2003,Origin_HousingPI25_2004,Origin_HousingPI25_2005,Origin_HousingPI25_2006,Origin_HousingPI25_2007,Origin_HousingPI25_2008,Origin_HousingPI25_2009,Origin_HousingPI25_2010,Origin_HousingPI25_2011,Origin_HousingPI25_2012,Origin_HousingPI25_2013,Origin_HousingPI25_2014,Origin_HousingPI25_2015,Origin_HousingPI25_2016,Origin_HousingPI25_2017,Origin_HousingPI25_2018,Origin_HousingPI25_2019,Origin_HousingPI25_2020,Origin_HousingPI25_2021,Origin_HousingPI25_2022,Origin_HousingPI25_2023,Origin_HousingPI25_2024,Origin_Labor_Force_2015,Origin_Labor_Force_2016,Origin_Labor_Force_2017,Origin_Labor_Force_2018,Origin_Labor_Force_2019,Origin_Labor_Force_2020,Origin_Labor_Force_2021,Origin_Labor_Force_2022,Origin_Labor_Force_2023,Origin_Labor_Force_2024,Origin_Employed_2015,Origin_Employed_2016,Origin_Employed_2017,Origin_Employed_2018,Origin_Employed_2019,Origin_Employed_2020,Origin_Employed_2021,Origin_Employed_2022,Origin_Employed_2023,Origin_Employed_2024,Origin_Unemployed_2015,Origin_Unemployed_2016,Origin_Unemployed_2017,Origin_Unemployed_2018,Origin_Unemployed_2019,Origin_Unemployed_2020,Origin_Unemployed_2021,Origin_Unemployed_2022,Origin_Unemployed_2023,Origin_Unemployed_2024,Origin_MSA_Name,Origin_Current_Residence_Population,Origin_Current_Residence_Population_MOE

# Step 0. Preprocessing

In [6]:
flow_data['Origin_loss_ratio'] = flow_data['Origin_climate_loss_total'] / flow_data['Origin_climate_exposure_total']
flow_data['Destination_loss_ratio'] = flow_data['Destination_climate_loss_total'] / flow_data['Destination_climate_exposure_total']

# 1: Within, 0: Between 
flow_data["within_region"] = (flow_data["Origin_Region"] == flow_data["Destination_Region"]).astype(int)

flow_data["Flow_rate"] = 100 * (flow_data["Flow"] / flow_data["Origin_Current_Residence_Population"] )

# Step1. Select Variables

In [7]:
## Select variables:
# y
flow_col      = "Flow"  # OD migration count

# x1
dep_risk_col  = "Origin_loss_ratio"
arr_risk_col  = "Destination_loss_ratio"   # e.g., Destination_loss_bldg_total or *_EALB / *_EALPE

# x2
dep_hpi_col   = "Origin_HousingPI25_2016_2020_mean"
arr_hpi_col   = "Destination_HousingPI25_2016_2020_mean"

# x3
dep_ue_col    = "Origin_avg_unemployment_rate_16_20"
arr_ue_col    = "Destination_avg_unemployment_rate_16_20"

# x4
move_type = "within_region"

# x5
origin_region = "Origin_Region"
destination_region = "Destination_Region"


train_cols = [flow_col, origin_region, destination_region, dep_risk_col, arr_risk_col, dep_hpi_col, arr_hpi_col, dep_ue_col, arr_ue_col, move_type]

In [8]:
flow_data[train_cols].sample(3)

,Flow,Origin_Region,Destination_Region,Origin_loss_ratio,Destination_loss_ratio,Origin_HousingPI25_2016_2020_mean,Destination_HousingPI25_2016_2020_mean,Origin_avg_unemployment_rate_16_20,Destination_avg_unemployment_rate_16_20,within_region
39165,3.50,Midwest,Midwest,0.00,0.00,3.42,2.38,3.76,6.35,1
4153,141.00,South,South,0.00,0.00,4.56,3.54,3.82,4.33,1
70871,718.00,West,West,0.00,0.00,3.94,4.42,2.65,3.38,1


# Step 2. Delta

#### Delta features for Regression
- dRisk      : risk(dest) - risk(origin); standardized to interpret per-σ effect
- dHousing   : P/I(dest) - P/I(origin);
- dUnemp     : UnempRate(dest) - UnempRate(origin)

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df = flow_data.copy()

# x1 - dRisk
df["dRisk"] = (df[arr_risk_col].astype(float) - df[dep_risk_col].astype(float)) 

# x2 - dHousing
df["dHousing"] = df[arr_hpi_col].astype(float) - df[dep_hpi_col].astype(float)

# x3 - dUnemp
df["dUnemp"]  = df[arr_ue_col].astype(float) - df[dep_ue_col].astype(float)

# Eliminate NaN value
need = [flow_col, "dRisk", "dHousing", "dUnemp"]

print(len(df['Origin_Code'].unique()), len(df['Destination_Code'].unique()))

df = df.dropna(subset=need).copy()
print(len(df['Origin_Code'].unique()), len(df['Destination_Code'].unique()))


# Standardization (z-score)
df["dRisk_z"] = scaler.fit_transform(df[["dRisk"]])


# in case consisder population size:
# log size variable:
df["log_pop_origin"] = np.log(df["Origin_Current_Residence_Population"] + 1)
df["log_pop_dest"]   = np.log(df["Destination_Current_Residence_Population"] + 1)

392 392
367 367


## z-dRisk Variable check

In [10]:
# dRisk basic statistics
mean_val = df["dRisk"].mean()
std_val  = df["dRisk"].std()
min_val  = df["dRisk"].min()
max_val  = df["dRisk"].max()

sigma_range = (max_val - min_val) / std_val

print("📊 dRisk Statistics")
print(f"mean: {mean_val:.6f}")
print(f"standard deviation: {std_val:.6f}")
print(f"min: {min_val:.6f}")
print(f"max: {max_val:.6f}")
print(f"min~max, total std: {sigma_range:.2f} σ")


📊 dRisk Statistics
mean: 0.000000
standard deviation: 0.000003
min: -0.000011
max: 0.000011
min~max, total std: 7.45 σ


In [11]:
df[['dRisk','dRisk_z']].sample(10)

,dRisk,dRisk_z
10505,-0.00,-2.10
60538,-0.00,-0.42
67726,-0.00,-1.68
2802,0.00,0.01
25358,-0.00,-0.72
80232,0.00,0.17
6892,-0.00,-2.03
63295,0.00,0.04
76480,-0.00,-0.59
12405,-0.00,-0.07


## Step 3. Possible Model Comparison

### 3.1. Set Formula

In [12]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete.count_model import ZeroInflatedPoisson, ZeroInflatedNegativeBinomialP
import pandas as pd

# model 1
formula_base = f"""{flow_col} ~ dRisk_z 
                           + dHousing
                           + dUnemp
                           """

# model 2
formula_move_type = f"""{flow_col} ~ dRisk_z * within_region
                           + dHousing * within_region
                           + dUnemp * within_region
                           """

# model 3
formula_including_origin = f"""{flow_col} ~ dRisk_z * within_region * C(Origin_Region)
                           + dHousing * within_region * C(Origin_Region)
                           + dUnemp * within_region * C(Origin_Region)
                           """

# model 4
formula_origin_destination = f"""{flow_col} ~ (dRisk_z + dHousing + dUnemp)
        * C(Origin_Region)
        * C(Destination_Region)"""


### 3.2 Formula selection

In [13]:
# formula Selection
formula = formula_origin_destination

### 3.3 Model Comparison

#### Choose among: Poisson / Negative Binomial / Linear Regression 

In [14]:
mean_y = df[flow_col].mean()
var_y = df[flow_col].var()
print("Variance-to-mean ratio:", var_y / mean_y)

Variance-to-mean ratio: 3137.7564597742125


In [15]:
results = {}

# 1) Poisson
results["Poisson"] = smf.glm(formula=formula, data=df, family=sm.families.Poisson()).fit()

# 2) Negative Binomial
results["NegBin"] = smf.glm(formula=formula, data=df, family=sm.families.NegativeBinomial()).fit()

# 3) Linear (Gaussian)
results["Linear"] = smf.glm(formula=formula, data=df, family=sm.families.Gaussian()).fit()


# comparison
comp = pd.DataFrame({
    "Model": list(results.keys()),
    "AIC": [res.aic for res in results.values()],
    "LogLik": [res.llf for res in results.values()]
}).sort_values("AIC")

# Best model select
best_model_name = comp.iloc[0]["Model"]

print("📊 Model Comparison (lower AIC better):")
print(comp)

print(f"\n✅ Best model selected (by AIC): {best_model_name}")


📊 Model Comparison (lower AIC better):
     Model           AIC         LogLik
1   NegBin    890,079.95    -444,975.98
2   Linear  1,274,311.56    -637,091.78
0  Poisson 29,541,377.72 -14,770,624.86

✅ Best model selected (by AIC): NegBin


## Step 4. Run the Best Model

In [16]:
best_res = results[best_model_name]

# --- Pring Result ---
print(best_res.summary())
if best_model_name in ["Poisson", "NegBin"]:
    coef = best_res.params
    se = best_res.bse
    pval = best_res.pvalues
    irr = np.exp(coef)

    result_table = pd.DataFrame({
        "coef": coef,
        "IRR": irr,
        "SE": se,
        "pval": pval
    })
    print("\n📑 Regression Results (coef, IRR, SE, p-value):")
    print(print(result_table.to_string()))
else:
    coef = best_res.params
    se = best_res.bse
    pval = best_res.pvalues

    result_table = pd.DataFrame({
        "coef": coef,
        "SE": se,
        "pval": pval
    })
    print("\n Regression Results (Gaussian family):")
    print(print(result_table.to_string()))


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   Flow   No. Observations:                81906
Model:                            GLM   Df Residuals:                    81842
Model Family:        NegativeBinomial   Df Model:                           63
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.4498e+05
Date:                Sat, 25 Oct 2025   Deviance:                   3.3205e+05
Time:                        19:49:43   Pearson chi2:                 8.89e+05
No. Iterations:                    23   Pseudo R-squ. (CS):             0.4067
Covariance Type:            nonrobust                                         
                                                                                coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------

## Step 5. Add more interpretability

Interpretability Module: IRR (Incident Rate Ratio) Breakdown
------------------------------------------------------------

Purpose:
- Convert regression coefficients (log-scale) into interpretable IRRs (exp(coef))
- Decompose results by Origin/Destination region and Within/Cross-region movement type
- Enable region- and flow-specific interpretation of marginal effects

Functions:
1. irr_by_origin_within(var)
   - Computes IRR for each combination of Origin Region × Within/Cross-region flow
   - Adjusts for interaction terms (Origin_Region × within_region)
   - Useful when model includes both Origin and within-region interactions

2. irr_by_within(var)
   - Computes IRR aggregated only by Within vs Cross-region movement
   - Simpler summary when no region-specific interactions are modeled

3. irr_by_origin_dest(var, coef)
   - Computes IRR for all Origin × Destination region pairs
   - Incorporates Origin, Destination, and their interaction effects
   - Produces a matrix-like IRR summary (OD interpretation)

Outputs:
- 'pivot' tables summarizing IRRs by flow type or region combination
- Facilitates interpretation of how ΔRisk, ΔHousing, and ΔUnemp affect migration
  across different regional patterns (e.g., Cross vs Within, Midwest vs South)

Interpretation:
- IRR > 1: positive association (↑ migration flow)
- IRR < 1: negative association (↓ migration flow)
- Region and within/cross distinctions clarify heterogeneity in response to Δ variables

In [17]:
def irr_by_origin_within(var):
    """Compute IRR for each Origin x Within-region combination for the given variable"""
    data = []
    for origin in ["Midwest", "Northeast", "South", "West"]:
        for within in [0, 1]:
            c = coef[var]
            if origin != "Midwest":
                c += coef.get(f"{var}:C(Origin_Region)[T.{origin}]", 0)
            if within == 1:
                c += coef.get(f"{var}:within_region", 0)
                if origin != "Midwest":
                    c += coef.get(f"{var}:within_region:C(Origin_Region)[T.{origin}]", 0)
            data.append({
                "Origin": origin,
                "within_region": "Within" if within else "Cross",
                "Variable": var,
                "IRR": np.exp(c)
            })
    return pd.DataFrame(data)

# --- helper: Compute IRR by Cross vs Within region ---
def irr_by_within(var):
    """Compute IRR by within-region status"""
    data = []
    for within in [0, 1]:
        c = coef[var]
        if within == 1:
            c += coef.get(f"{var}:within_region", 0)
        data.append({
            "within_region": "Within" if within else "Cross",
            "Variable": var,
            "IRR": np.exp(c)
        })
    return pd.DataFrame(data)

def irr_by_origin_dest(var, coef):
    """Compute IRR for each Origin × Destination region combination for the given variable"""
    data = []
    origins = ["Midwest", "Northeast", "South", "West"]
    dests   = ["Midwest", "Northeast", "South", "West"]
    
    for o in origins:
        for d in dests:
            c = coef.get(var, 0)
            # 1. Origin effect
            if o != "Midwest":
                c += coef.get(f"{var}:C(Origin_Region)[T.{o}]", 0)
            # 2. Destination effect
            if d != "Midwest":
                c += coef.get(f"{var}:C(Destination_Region)[T.{d}]", 0)
            # 3. Origin × Destination interaction
            if (o != "Midwest") and (d != "Midwest"):
                c += coef.get(f"{var}:C(Origin_Region)[T.{o}]:C(Destination_Region)[T.{d}]", 0)
            data.append({
                "Origin": o,
                "Destination": d,
                var: np.exp(c)
            })
    return pd.DataFrame(data)

# interpretation - friednly reshaping
if formula == formula_including_origin:
    print(1)
    coef = result_table["coef"]
    
    summary = pd.concat([
        irr_by_origin_within("dRisk_z"),
        irr_by_origin_within("dHousing"),
        irr_by_origin_within("dUnemp")
    ])
    
    # pivot for better readability
    pivot = summary.pivot(index=["Origin","within_region"], columns="Variable", values="IRR").round(2)

elif formula == formula_move_type:  
    # --- interpretation-friendly reshaping ---
    coef = result_table["coef"]
    
    summary = pd.concat([
        irr_by_within("dRisk_z"),
        irr_by_within("dHousing"),
        irr_by_within("dUnemp")
    ])
    # pivot for better readability
    pivot = summary.pivot(index="within_region", columns="Variable", values="IRR").round(2)
    pivot = pivot.reset_index()
    
    print("\n📊 Interpretation-friendly IRR summary (Cross vs Within):")
    print(pivot)
    
elif formula == formula_origin_destination: 
    # --- executiin ---
    coef = result_table["coef"]
    
    irr_risk    = irr_by_origin_dest("dRisk_z", coef)
    irr_housing = irr_by_origin_dest("dHousing", coef)
    irr_unemp   = irr_by_origin_dest("dUnemp", coef)
    
    pivot = irr_risk.merge(irr_housing, on=["Origin","Destination"]).merge(irr_unemp, on=["Origin","Destination"])
    pivot = pivot.round(2)
    
    print("\n📊 Interpretation-friendly OD IRR summary:")


📊 Interpretation-friendly OD IRR summary:


## Step 6. Regression Result

In [18]:
print(formula)

Flow ~ (dRisk_z + dHousing + dUnemp)
        * C(Origin_Region)
        * C(Destination_Region)


In [19]:
pivot

,Origin,Destination,dRisk_z,dHousing,dUnemp
0,Midwest,Midwest,0.93,0.94,0.90
1,Midwest,Northeast,1.77,1.09,0.87
2,Midwest,South,1.27,0.79,0.70
3,Midwest,West,1.16,0.96,0.86
4,Northeast,Midwest,0.20,0.83,1.10
5,Northeast,Northeast,0.97,0.88,0.94
6,Northeast,South,1.22,0.73,0.76
7,Northeast,West,2.01,0.86,0.81
8,South,Midwest,0.91,1.20,1.31
9,South,Northeast,1.11,1.15,1.09


## Next Step: Shapley Value calculation